## Etkileşimli Phi 3 Mini 4K Talimatlı Sohbet Botu ile Whisper

### Giriş:
Etkileşimli Phi 3 Mini 4K Talimatlı Sohbet Botu, kullanıcıların Microsoft Phi 3 Mini 4K talimatlı demo ile metin veya sesli giriş kullanarak etkileşimde bulunmasını sağlayan bir araçtır. Sohbet botu, çeviri, hava durumu güncellemeleri ve genel bilgi toplama gibi çeşitli görevler için kullanılabilir.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Huggingface Erişim Jetonunuzu Oluşturun](https://huggingface.co/settings/tokens)

Yeni bir jeton oluşturun  
Yeni bir ad verin  
Yazma izinlerini seçin  
Jetonu kopyalayın ve güvenli bir yerde saklayın


Aşağıdaki Python kodu iki ana görevi yerine getirir: `os` modülünü içe aktarmak ve bir ortam değişkeni ayarlamak.

1. `os` modülünü içe aktarmak:
   - Python'daki `os` modülü, işletim sistemiyle etkileşim kurmanın bir yolunu sağlar. Ortam değişkenlerine erişmek, dosya ve dizinlerle çalışmak gibi çeşitli işletim sistemiyle ilgili görevleri gerçekleştirmenize olanak tanır.
   - Bu kodda, `os` modülü `import` ifadesi kullanılarak içe aktarılmıştır. Bu ifade, `os` modülünün işlevselliğini mevcut Python betiğinde kullanılabilir hale getirir.

2. Bir ortam değişkeni ayarlamak:
   - Ortam değişkeni, işletim sisteminde çalışan programlar tarafından erişilebilen bir değerdir. Birden fazla program tarafından kullanılabilecek yapılandırma ayarlarını veya diğer bilgileri depolamanın bir yoludur.
   - Bu kodda, yeni bir ortam değişkeni `os.environ` sözlüğü kullanılarak ayarlanıyor. Sözlüğün anahtarı `'HF_TOKEN'` ve değeri `HUGGINGFACE_TOKEN` değişkeninden atanıyor.
   - `HUGGINGFACE_TOKEN` değişkeni, bu kod parçasının hemen üstünde tanımlanmıştır ve `"hf_**************"` gibi bir dize değeri `#@param` sözdizimi kullanılarak atanmıştır. Bu sözdizimi genellikle Jupyter defterlerinde, kullanıcı girdisi ve parametre yapılandırmasını doğrudan defter arayüzünde sağlamak için kullanılır.
   - `'HF_TOKEN'` ortam değişkeni ayarlandığında, programın diğer bölümleri veya aynı işletim sisteminde çalışan diğer programlar tarafından erişilebilir hale gelir.

Genel olarak, bu kod `os` modülünü içe aktarır ve `HUGGINGFACE_TOKEN` değişkeninde sağlanan değerle `'HF_TOKEN'` adlı bir ortam değişkeni ayarlar.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Bu kod parçası, Jupyter Notebook veya IPython'daki mevcut hücrenin çıktısını temizlemek için kullanılan clear_output adlı bir fonksiyonu tanımlar. Kodun işlevselliğini anlamak için parçalayalım:

clear_output fonksiyonu, wait adlı bir parametre alır. Bu parametre bir boolean (doğru/yanlış) değerdir. Varsayılan olarak, wait False olarak ayarlanmıştır. Bu parametre, fonksiyonun mevcut çıktıyı temizlemeden önce yeni bir çıktı mevcut olana kadar bekleyip beklemeyeceğini belirler.

Fonksiyonun kendisi, mevcut hücrenin çıktısını temizlemek için kullanılır. Jupyter Notebook veya IPython'da bir hücre çıktı ürettiğinde, örneğin yazdırılmış metin veya grafiksel çizimler gibi, bu çıktı hücrenin altında görüntülenir. clear_output fonksiyonu, bu çıktıyı temizlemenize olanak tanır.

Kod parçasında fonksiyonun uygulanışı verilmemiştir, bu durum üç nokta (...) ile belirtilmiştir. Üç nokta, çıktıyı temizleme işlemini gerçekleştiren gerçek kodun yerine bir yer tutucu görevi görür. Fonksiyonun uygulanışı, mevcut hücreden çıkışı kaldırmak için Jupyter Notebook veya IPython API'si ile etkileşim kurmayı içerebilir.

Genel olarak, bu fonksiyon, Jupyter Notebook veya IPython'daki mevcut hücrenin çıktısını temizlemek için kullanışlı bir yöntem sunar. Bu, etkileşimli kodlama oturumları sırasında görüntülenen çıktıyı yönetmeyi ve güncellemeyi kolaylaştırır.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Edge TTS hizmetini kullanarak metinden konuşmaya (TTS) işlemi gerçekleştirin. İlgili fonksiyonların uygulamalarını tek tek inceleyelim:

1. `calculate_rate_string(input_value)`: Bu fonksiyon, bir giriş değeri alır ve TTS sesi için hız dizesini hesaplar. Giriş değeri, konuşmanın istenen hızını temsil eder; 1 değeri normal hızı ifade eder. Fonksiyon, giriş değerinden 1 çıkarır, 100 ile çarpar ve giriş değerinin 1'e eşit veya büyük olup olmadığına göre işareti belirler. Fonksiyon, "{sign}{rate}" formatında hız dizesini döndürür.

2. `make_chunks(input_text, language)`: Bu fonksiyon, bir giriş metni ve bir dil parametresi alır. Giriş metnini dilin özel kurallarına göre parçalara böler. Bu uygulamada, dil "English" ise, fonksiyon metni her nokta (".") ile böler ve baştaki veya sondaki boşlukları kaldırır. Daha sonra her parçaya bir nokta ekler ve filtrelenmiş parça listesini döndürür.

3. `tts_file_name(text)`: Bu fonksiyon, giriş metnine dayanarak TTS ses dosyası için bir dosya adı oluşturur. Metin üzerinde birkaç dönüşüm gerçekleştirir: sondaki noktayı (varsa) kaldırır, metni küçük harfe çevirir, baştaki ve sondaki boşlukları temizler ve boşlukları alt çizgi ile değiştirir. Daha sonra metni maksimum 25 karaktere kadar kısaltır (eğer daha uzunsa) veya metin boşsa tamamını kullanır. Son olarak, [`uuid`] modülünü kullanarak rastgele bir dize oluşturur ve kısaltılmış metinle birleştirerek "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3" formatında dosya adını oluşturur.

4. `merge_audio_files(audio_paths, output_path)`: Bu fonksiyon, birden fazla ses dosyasını tek bir ses dosyasında birleştirir. Bir ses dosyası yolu listesi ve bir çıktı yolu parametresi alır. Fonksiyon, [`merged_audio`] adlı boş bir `AudioSegment` nesnesi başlatır. Daha sonra her ses dosyası yolunu iterasyonla işler, `pydub` kütüphanesinden `AudioSegment.from_file()` yöntemiyle ses dosyasını yükler ve mevcut ses dosyasını [`merged_audio`] nesnesine ekler. Son olarak, birleştirilmiş sesi belirtilen çıktı yoluna MP3 formatında dışa aktarır.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Bu fonksiyon, Edge TTS hizmetini kullanarak TTS işlemini gerçekleştirir. Bir metin parçası listesi, konuşma hızı, ses adı ve kaydetme yolu parametrelerini alır. Parça sayısı 1'den büyükse, fonksiyon bireysel parça ses dosyalarını depolamak için bir dizin oluşturur. Daha sonra her parçayı iterasyonla işler, `calculate_rate_string()` fonksiyonunu, ses adını ve parça metnini kullanarak bir Edge TTS komutu oluşturur ve `os.system()` fonksiyonuyla komutu çalıştırır. Komut başarıyla çalıştırılırsa, oluşturulan ses dosyasının yolunu bir listeye ekler. Tüm parçalar işlendiğinde, bireysel ses dosyalarını `merge_audio_files()` fonksiyonuyla birleştirir ve birleştirilmiş sesi belirtilen kaydetme yoluna kaydeder. Eğer sadece bir parça varsa, doğrudan Edge TTS komutunu oluşturur ve sesi kaydetme yoluna kaydeder. Son olarak, oluşturulan ses dosyasının kaydetme yolunu döndürür.

6. `random_audio_name_generate()`: Bu fonksiyon, [`uuid`] modülünü kullanarak rastgele bir ses dosyası adı oluşturur. Rastgele bir UUID oluşturur, bunu bir dizeye çevirir, ilk 8 karakterini alır, ".mp3" uzantısını ekler ve rastgele ses dosyası adını döndürür.

7. `talk(input_text)`: Bu fonksiyon, TTS işlemini gerçekleştirmek için ana giriş noktasıdır. Bir giriş metni parametresi alır. Öncelikle giriş metninin uzunluğunu kontrol eder ve uzun bir cümle olup olmadığını (600 karakterden büyük veya eşit) belirler. Uzunluğa ve `translate_text_flag` değişkeninin değerine bağlı olarak dili belirler ve `make_chunks()` fonksiyonunu kullanarak metin parçaları listesini oluşturur. Daha sonra ses dosyası için bir kaydetme yolu oluşturmak üzere `random_audio_name_generate()` fonksiyonunu çağırır. Son olarak, TTS işlemini gerçekleştirmek için `edge_free_tts()` fonksiyonunu çağırır ve oluşturulan ses dosyasının kaydetme yolunu döndürür.

Genel olarak, bu fonksiyonlar giriş metnini parçalara bölmek, ses dosyası için bir dosya adı oluşturmak, Edge TTS hizmetini kullanarak TTS işlemini gerçekleştirmek ve bireysel ses dosyalarını tek bir ses dosyasında birleştirmek için birlikte çalışır.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

İki fonksiyonun uygulanması: convert_to_text ve run_text_prompt, ayrıca iki sınıfın tanımı: str ve Audio.

convert_to_text fonksiyonu, bir audio_path girdisi alır ve whisper_model adlı bir model kullanarak sesi metne dönüştürür. Fonksiyon, öncelikle gpu bayrağının True olarak ayarlanıp ayarlanmadığını kontrol eder. Eğer True ise, whisper_model belirli parametrelerle kullanılır: word_timestamps=True, fp16=True, language='English', ve task='translate'. Eğer gpu bayrağı False ise, whisper_model fp16=False ile kullanılır. Ortaya çıkan transkripsiyon 'scan.txt' adlı bir dosyaya kaydedilir ve metin olarak geri döndürülür.

run_text_prompt fonksiyonu, bir message ve chat_history girdisi alır. Bu fonksiyon, phi_demo fonksiyonunu kullanarak girdiye dayalı olarak bir chatbot'tan yanıt oluşturur. Oluşturulan yanıt daha sonra talk fonksiyonuna iletilir, bu yanıtı bir ses dosyasına dönüştürür ve dosya yolunu geri döndürür. Audio sınıfı, ses dosyasını görüntülemek ve oynatmak için kullanılır. Ses, IPython.display modülünden display fonksiyonu kullanılarak görüntülenir ve Audio nesnesi autoplay=True parametresi ile oluşturulur, böylece ses otomatik olarak çalmaya başlar. chat_history, girdiyi ve oluşturulan yanıtı içerecek şekilde güncellenir ve boş bir string ile güncellenmiş chat_history geri döndürülür.

str sınıfı, Python'da karakter dizilerini temsil eden yerleşik bir sınıftır. Bu sınıf, capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill ve daha fazlası gibi çeşitli yöntemler sağlar. Bu yöntemler, arama, değiştirme, biçimlendirme ve dizilerle çalışma gibi işlemleri gerçekleştirmenize olanak tanır.

Audio sınıfı, bir ses nesnesini temsil eden özel bir sınıftır. Jupyter Notebook ortamında bir ses oynatıcı oluşturmak için kullanılır. Sınıf, data, filename, url, embed, rate, autoplay ve normalize gibi çeşitli parametreleri kabul eder. data parametresi bir numpy array, örneklerin bir listesi, bir dosya adı veya URL'yi temsil eden bir string ya da ham PCM verisi olabilir. filename parametresi, ses verilerini yüklemek için yerel bir dosya belirtmek için kullanılır ve url parametresi, ses verilerini indirmek için bir URL belirtmek için kullanılır. embed parametresi, ses verilerinin bir veri URI kullanılarak mı yoksa orijinal kaynaktan mı referans alınacağını belirler. rate parametresi, ses verilerinin örnekleme oranını belirtir. autoplay parametresi, sesin otomatik olarak çalınıp çalınmayacağını belirler. normalize parametresi, ses verilerinin maksimum olası aralığa yeniden ölçeklenip ölçeklenmeyeceğini belirtir. Audio sınıfı ayrıca reload gibi ses verilerini dosya veya URL'den yeniden yüklemek için yöntemler ve src_attr, autoplay_attr ve element_id_attr gibi HTML'deki ses öğesi için ilgili özellikleri almak için öznitelikler sağlar.

Genel olarak, bu fonksiyonlar ve sınıflar, sesi metne dönüştürmek, bir chatbot'tan sesli yanıtlar oluşturmak ve Jupyter Notebook ortamında sesleri görüntülemek ve oynatmak için kullanılır.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalardan sorumlu değiliz.
